Database operations to build the structure for visualization

I will need 4 groups of data:
* topics (aggregator)
* docs
* words
* people

In [1]:
import pandas as pd
import json
import pickle
import re
import math
import gensim
from collections import Counter

In [2]:
outputs = '../outputs/'

In [3]:
file_tables = '../outputs/tables_dict.pkl'
cpdoc_tables = pickle.load(open(file_tables, 'rb'))

tests

In [4]:
file_tables_v2 = '../outputs/tables_dict-version201906.pkl'
cpdoc_tables_v2 = pickle.load(open(file_tables, 'rb'))

In [5]:
for table in cpdoc_tables:
    print('table:', table, 'length:', len(cpdoc_tables[table]))

table: person_doc  length: 10754
table: top_persons length: 1081
table: persons length: 1943
table: top_countries length: 227
table: docs length: 10279
table: top_topics length: 10
table: doc_counts  length: 71
table: countries length: 278
table: top_classifications length: 0
table: topic_doc length: 97701
table: topics length: 100
table: country_doc length: 39160


tests end

In [6]:
list(cpdoc_tables)

['person_doc ',
 'top_persons',
 'persons',
 'top_countries',
 'docs',
 'top_topics',
 'doc_counts ',
 'countries',
 'top_classifications',
 'topic_doc',
 'topics',
 'country_doc']

# docs and topics

In [7]:
topic_doc = cpdoc_tables['topic_doc']
topic_doc.head()

,doc_id,topic_id,topic_score,date
0,ag_1973.11.20_doc_I-100,5009,0.167579,1974-02-22
1,ag_1973.11.20_doc_I-100,5010,0.040318,1974-02-22
2,ag_1973.11.20_doc_I-100,5040,0.092381,1974-02-22
3,ag_1973.11.20_doc_I-100,5041,0.090368,1974-02-22
4,ag_1973.11.20_doc_I-100,5051,0.029285,1974-02-22


# docs and persons

In [8]:
persons = cpdoc_tables['persons']
persons = persons.rename(index=str, columns={"id": "person_id"})

In [9]:
person_doc = cpdoc_tables['person_doc ']
person_doc = pd.merge(person_doc, persons, on='person_id', how='inner')
person_doc = person_doc.drop(columns=['date', 'source', 'birth_year', 'death_year', 'description'])
person_doc = person_doc.replace('(.*),(.*),.*', r'\1,\2', regex=True)
person_doc = person_doc.replace('(.*), (.*)', r'\2 \1', regex=True)

In [10]:
#remove special characters
person_doc = person_doc.name.apply(pd.Series) \
    .replace('[“”]', '', regex=True) \
    .replace('[áàãâ]', 'a', regex=True) \
    .replace('[óòõô]', 'o', regex=True) \
    .replace('[éèê]', 'e', regex=True) \
    .replace('[íì]', 'i', regex=True) \
    .replace('[úù]', 'u', regex=True) \
    .replace('ç', 'c', regex=True) \
    .merge(person_doc, left_index = True, right_index = True) \
    .drop("name", axis=1) \
    .rename(columns={0: "name"})

In [11]:
person_doc.head()

,name,person_id,doc_id,person_count
0,Dean Gooderham Acheson,100070,ag_1977.07.00_doc_I-4,1
1,John Quincy Adams,100097,be_1974.03.15_doc_XI-2,1
2,Konrad Adenauer,100118,be_1974.03.15_doc_X-21,1
3,Konrad Adenauer,100118,be_1977.06.01_doc_II-28,1
4,Konrad Adenauer,100118,pn_1974.08.15_doc_I-3,1


In [12]:
person_doc_count = pd.DataFrame(person_doc.groupby(['name'])['person_count'].agg('count')) #.apply(sum)
person_doc_count = person_doc_count.reset_index()

# docs and tokens

In [13]:
docs = cpdoc_tables['docs'][['id', 'title', 'pdf', 'body']]
docs = docs.rename(index=str, columns={"id": "doc_id"})
docs['length'] = docs['body'].apply(lambda x: len(x.split()))
docs.head()

,doc_id,title,pdf,body,length
0,ag_1973.11.20_doc_I-1,Documentos felicitando Azeredo da Silveira pel...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\njr, /h ' ""& m$ hb'h'qo\n& la'?\n\n, é %%3...",261
1,ag_1973.11.20_doc_I-100,Documentos felicitando Azeredo da Silveira pel...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"mjm h\n\n. . . ' "" . ww and\n& !\nzczc tea363 ...",173
2,ag_1973.11.20_doc_I-101,Documentos felicitando Azeredo da Silveira pel...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n \n \n\n \n\n \n\n114"":\n\n \n\n ...",72
3,ag_1973.11.20_doc_I-102,Documentos felicitando Azeredo da Silveira pel...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\n"" "", ""m-nmfvww mmm/""ar? &"", wa "" ""***-**""...",131
4,ag_1973.11.20_doc_I-103,Documentos felicitando Azeredo da Silveira pel...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,\n\n \n\nmi*/gm\n%adazw/sõ'\n\n%% && %%%&\n\n...,295


# words and topics

In [14]:
computer = input("home OR fgv OR broken? ") 
if computer == 'home': outer_inputs = 'D:/pseudo-dropbox/backups-fgv/azeredo_lda/'
elif computer == 'broken': outer_inputs = 'C:/Users/marcelo.old-note/Documents/'
else: print('computer is not defined')

home OR fgv OR broken? home


In [15]:
file_lda_model = outer_inputs+'model_lda_100_rs_00.pkl'
lda_model = pickle.load(open(file_lda_model, 'rb'))

In [16]:
def remove_special_char(text):
    text = re.sub('[áàãâ]', 'a', text)
    text = re.sub('[óòõô]', 'o', text)
    text = re.sub('[éèê]', 'e', text)
    text = re.sub('[íì]', 'i', text)
    text = re.sub('[úù]', 'u', text)
    text = re.sub('ç', 'c', text)
    return text

In [17]:
def retrieve_tokens(text, df):
    main_tokens = []
    for token in df['tokens']:
        if token in text:
            token = remove_special_char(token)
            main_tokens.append(token)
    return main_tokens

# build vis table
Topic: Nuclear Brazil

In [18]:
main_topics = [['International Cooperation for Development', 99], ['Geisel foreign policy: ideas and action', 61], 
               ['Brazilian government and private investment', 39], ['UN system', 63], 
               ['International Economic Relations of Brazil', 56], ['United States of America', 89], 
               ['Latin America and Caribbean', 4], ['Itaipu plant: technical discussions', 49], ['Nuclear Brazil', 7], 
               ['Brazil, Africa and decolonization', 45]]

In [50]:
def build_vis_table(keyword, df):
    for pair in main_topics:
        if pair[0] == keyword: topic_num = pair[1]    
    if len(str(topic_num)) == 1: topic_id = '500'+str(topic_num)
    else: topic_id = '50'+str(topic_num)
    topic_id = int(topic_id)
        
    topic_tokens = lda_model.print_topics(-1, num_words=20)[topic_num]
    #topic_tokens[1]
    
    tokens = []
    scores = []
    for i in topic_tokens[1].split('+'):
        token = re.sub('.*\*"(.*)".*', r'\1', i)
        score = re.sub(' *(.*)\*.*', r'\1', i)
        score = float(score)
        tokens.append(token)
        scores.append(score)
    token_score_dict = {'tokens': tokens, 'scores': scores}
    
    token_score = pd.DataFrame(token_score_dict)
    
    docs['tokens'] = docs['body'].apply(lambda text: retrieve_tokens(text, token_score))
    
    
    #filter by topic
    df = df.loc[df['topic_id'] == topic_id].sort_values(by=['topic_score'], ascending=False)
    df = df.head(20)
    
    ### merge topics and tokens
    df = pd.merge(df, docs, on='doc_id', how='inner')
    df = df.drop(columns=['title'])
    
    ### relate docs to list of persons
    #filters person_doc
    array = list(df['doc_id'])
    person_doc_filtered = person_doc.loc[person_doc['doc_id'].isin(array)]
    
    #names_list = [row['name'] for index, row in person_doc_filtered.iterrows()]
    #person_doc_filtered['name'] = names_list
    
    #apply list of persons
    person_doc_filtered = person_doc_filtered.groupby(['doc_id'])['name'].apply(list)
    person_doc_filtered = pd.DataFrame({'doc_id':person_doc_filtered.index, 'names':person_doc_filtered.values})
    
    #merge topics and persons
    df = pd.merge(df, person_doc_filtered, on='doc_id', how='outer')
    df['year'] = df['date'].apply(lambda x: x.year)
    for row in df.loc[df.names.isnull(), 'names'].index:
        df.at[row, 'names'] = []
    
    return df

In [51]:
topic_vis_dict = {}

In [52]:
for pair in main_topics:
    keyword = pair[0]
    topic_vis_dict[keyword] = build_vis_table(keyword, topic_doc)

In [53]:
topic_vis_dict.keys()

dict_keys(['International Cooperation for Development', 'Geisel foreign policy: ideas and action', 'Brazilian government and private investment', 'UN system', 'International Economic Relations of Brazil', 'United States of America', 'Latin America and Caribbean', 'Itaipu plant: technical discussions', 'Nuclear Brazil', 'Brazil, Africa and decolonization'])

In [36]:
topic_vis_dict['Nuclear Brazil'].tail()

,doc_id,topic_id,topic_score,date,pdf,body,length,tokens,names,year
15,pn_1974.08.15_doc_III-31,5007,0.303900,1978-01-15,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,secreto-excâusãvo\n\nsubsídios para as consult...,4549,"[nuclear, acordo, energia, nucleares, brasil, ...","[James Earl (Jimmy) Carter, Helmut Schmidt, Cy...",1978
16,pn_1976.12.28_doc_29,5007,0.300885,1978-01-15,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"telegrama recebido '\n\naaaa [97034 . é?\no"" '...",845,"[nuclear, acordo, energia, nucleares, brasil, ...",[James Earl (Jimmy) Carter],1978
17,pn_1974.08.15_doc_II-1,5007,0.290109,1976-01-15,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\nmmm\n\n""a. . .. , . ..\n\n' o acordo nucl...",1672,"[nuclear, acordo, energia, nucleares, brasil, ...",[James Earl (Jimmy) Carter],1976
18,pn_1976.12.28_doc_16,5007,0.286767,1978-01-06,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,antonio la'. aiii-1315230 da sxmrâxm\njáwówa i...,330,"[acordo, energia, brasil, uranio, rfa, salvagu...",[Antonio Azeredo da Silveira],1978
19,d_1974.04.23_doc_XXI-12,5007,0.285340,1979-06-15,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,\n\n \n\nlembrete n? 011\n\npolítica nuclear ...,1590,"[nuclear, acordo, energia, nucleares, brasil, ...",[James Earl (Jimmy) Carter],1979


In [56]:
topic_vis_dict['Itaipu plant: technical discussions'].tail()

,doc_id,topic_id,topic_score,date,pdf,body,length,tokens,name,year
15,bp_1977.03.10_doc_V-3,5049,0.471688,1978-01-15,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,". ""* aas_ manoa-to\n/ . _ - . . awm)\n\nsala-1...",6339,"[itaipu, rio, cota, operacao, energia, constru...","[Antonio Azeredo da Silveira, Costa Cavalcanti...",1978
16,bp_1977.09.13_doc_II-2,5049,0.469907,1977-04-15,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,- informação no? 332/76 - estudo sobre a fixaç...,2444,"[itaipu, rio, corpus, cota, operacao, energia,...",[],1977
17,d_1974.03.26_doc_XXXI-44,5049,0.466800,1978-01-15,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\n \n\n \n\nwm\n""mg . .\na"" rio âecreto\n...",6152,"[itaipu, rio, cota, operacao, energia, constru...","[Antonio Azeredo da Silveira, Costa Cavalcanti...",1978
18,bp_1977.09.13_doc_I-14,5049,0.455080,1977-04-15,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"/\n\n \n\n \n\n \n\na\n\n \n\n!\n\n \n\n o:""i...",1690,"[itaipu, rio, corpus, cota, construcao, parana...",[],1977
19,bp_1977.03.10_doc_III-18,5049,0.452564,1977-11-15,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\nwma-03.40\njg?\n(é. /, () h\n\n confidenc...",409,"[itaipu, rio, corpus, cota, operacao, construc...",[],1977


# pickle file

In [37]:
topics_dict_file = '../outputs/topics_dict.pkl'

pickle for general data already saved. To overwrite:
```python
pickle.dump(topic_vis_dict, open(topics_dict_file, 'wb'))
```

In [6]:
topic_vis_dict = pickle.load(open(topics_dict_file, 'rb'))

# Visualization of metadata

In [238]:
topics_metadata = pd.DataFrame(columns=['mean_score', 'mean_length'])
count=0
for key, df in topic_vis_dict.items():
    score_mean = df['topic_score'].mean()
    length_mean = df['length'].mean()
    row = pd.DataFrame([[key, score_mean, length_mean]], columns=['key', 'mean_score', 'mean_length'], index=[count])
    topics_metadata = topics_metadata.append(row)
    count+=1

In [249]:
topics_metadata.head(2)

,key,mean_length,mean_score
0,International Cooperation for Development,750.70,0.520749
1,Geisel foreign policy: ideas and action,1490.25,0.371362


In [246]:
topics_metadata_json = []
for index, row in topics_metadata.iterrows():
    temp_dict = {'key': row['key'], 'mean_length': row['mean_length'], 'mean_score': row['mean_score']}
    topics_metadata_json.append(temp_dict)

In [245]:
topics_metadata_json[:2]

[{'key': 'International Cooperation for Development',
  'mean_length': 750.7,
  'mean_score': 0.52074945},
 {'key': 'Geisel foreign policy: ideas and action',
  'mean_length': 1490.25,
  'mean_score': 0.37136150000000007}]

In [247]:
json.dump(topics_metadata_json, open('../outputs/topics_metadata.json', 'w'))

# Visualization of a specific topic

list split  in pandas: https://mikulskibartosz.name/how-to-split-a-list-inside-a-dataframe-cell-into-rows-in-pandas-9849d8ff2401

In [42]:
nuclear_brazil_df = topic_vis_dict['Nuclear Brazil']
nuclear_brazil_df['doc'] = nuclear_brazil_df['doc_id'].apply(lambda text: re.sub('.*(doc.*)', r'\1', text))
cols = nuclear_brazil_df.columns.tolist()
cols = cols[:3] + cols[4:] + cols[3:4]
nuclear_brazil_df = nuclear_brazil_df[cols]
nuclear_brazil_df.tail()

,doc_id,topic_id,topic_score,pdf,body,length,tokens,names,year,doc,date
15,pn_1974.08.15_doc_III-31,5007,0.303900,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,secreto-excâusãvo\n\nsubsídios para as consult...,4549,"[nuclear, acordo, energia, nucleares, brasil, ...","[James Earl (Jimmy) Carter, Helmut Schmidt, Cy...",1978,doc_III-31,1978-01-15
16,pn_1976.12.28_doc_29,5007,0.300885,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"telegrama recebido '\n\naaaa [97034 . é?\no"" '...",845,"[nuclear, acordo, energia, nucleares, brasil, ...",[James Earl (Jimmy) Carter],1978,doc_29,1978-01-15
17,pn_1974.08.15_doc_II-1,5007,0.290109,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\nmmm\n\n""a. . .. , . ..\n\n' o acordo nucl...",1672,"[nuclear, acordo, energia, nucleares, brasil, ...",[James Earl (Jimmy) Carter],1976,doc_II-1,1976-01-15
18,pn_1976.12.28_doc_16,5007,0.286767,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,antonio la'. aiii-1315230 da sxmrâxm\njáwówa i...,330,"[acordo, energia, brasil, uranio, rfa, salvagu...",[Antonio Azeredo da Silveira],1978,doc_16,1978-01-06
19,d_1974.04.23_doc_XXI-12,5007,0.285340,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,\n\n \n\nlembrete n? 011\n\npolítica nuclear ...,1590,"[nuclear, acordo, energia, nucleares, brasil, ...",[James Earl (Jimmy) Carter],1979,doc_XXI-12,1979-06-15


# build main data for each topic

In [80]:
topics = ['Nuclear Brazil', 'Itaipu plant: technical discussions']

In [14]:
dict_topics_json = {}

In [79]:
range(1,len(TOPICS)+1)

range(1, 3)

In [32]:
for topic,i in zip(topics, range(1,len(topics)+1)):
    topic_name = topic.lower()
    topic_name = re.sub('[\:\,]', '', topic_name)
    topic_name = re.sub('(^\w+ \w+).*', r'\1', topic_name)
    topic_name = re.sub('\s', r'_', topic_name)
    print(topic_name)
    dict_topics_json[topic_name] = topic_vis_dict[topic]
    dict_topics_json[topic_name]['doc'] = dict_topics_json[topic_name]['doc_id'].apply(lambda text: re.sub('.*(doc.*)', r'\1', text))
    cols = dict_topics_json[topic_name].columns.tolist()
    cols = cols[:3] + cols[4:] + cols[3:4]
    dict_topics_json[topic_name] = dict_topics_json[topic_name][cols]
    
    topic_transposed = dict_topics_json[topic_name] \
        .drop(["body", "topic_id"], axis = 1) \
        .rename(columns={"pdf": "url"}) \
        .transpose()
    
    # build json of main data
    topic_json = topic_transposed.to_json(orient='columns')
    topic_json = re.sub('"\d+":', '', topic_json)
    topic_json = re.sub("^{", "[", topic_json)
    topic_json = re.sub("}$", "]", topic_json)
    topic_json = json.loads(topic_json)
    
    # save json
    json_file = 'topic_{}.json'.format(str(i))
    file_path = outputs+json_file
    json.dump(topic_json, open(file_path, 'w'))

nuclear_brazil
itaipu_plant


# define function to melt dataframes

In [40]:
def get_melted_df(df,variable):
    cols_to_drop = set(df.columns) - set(['doc_id'])
    topic = df[variable].apply(pd.Series) \
        .merge(df, left_index = True, right_index = True) \
        .drop(cols_to_drop, axis = 1) \
        .melt(id_vars = ['doc_id'], value_name = variable) \
        .drop("variable", axis = 1) \
        .dropna()
    return topic

# generate json files related to persons
Get connections between docs and persons. Those represent the edges of a graph.

In [81]:
for topic,i in zip(topics, range(1,len(topics)+1)):
    #build list of docs for each person
    melted_df = get_melted_df(topic_vis_dict[topic], 'names')
    melted_df = melted_df.groupby(['names'])['doc_id'].apply(list)
    melted_df = pd.DataFrame({'name':melted_df.index, 'docs':melted_df.values})
    
    # build json of list of names
    names_list = []
    for index, row in melted_df.iterrows():
        name = row['name']
        name_count = math.log(int(person_doc_count.loc[person_doc_count['name'] == name]['person_count']),1.5)
        docs = row['docs']
        
        temp_dict = {'name': name, 'count': name_count, 'docs': docs}
        names_list.append(temp_dict)
        
    # save json
    file_path = outputs+'names_list_{}.json'.format(str(i))
    json.dump(names_list, open(file_path, 'w'))

# generate json files related to tokens
Get connections between docs and tokens. Those represent the edges of a graph.

In [96]:
for topic,i in zip(topics, range(1,len(topics)+1)):
    #build list of docs for each token
    melted_df = get_melted_df(topic_vis_dict[topic], 'tokens')
    melted_df = melted_df.groupby(['tokens'])['doc_id'].apply(list)
    melted_df = pd.DataFrame({'token':melted_df.index, 'docs':melted_df.values})
    
    #get data from lda_model
    for pair in main_topics:
        if pair[0] == topic: topic_num = pair[1]
    topic_tokens = lda_model.print_topics(-1, num_words=20)[topic_num]
    topic_tokens = topic_tokens[1].split('+')
    
    #topic_tokens will be a list with elements such as: '0.085*"nuclear" ',
    tokens_list = []
    for pair in topic_tokens:
        #get tokens and scores
        token = re.sub('.*\*"(.*)".*', r'\1', pair)
        token = remove_special_char(token)
        score = re.sub(' *(.*)\*.*', r'\1', pair)
        score = float(score)

        #remove tokens irrelevantes/problematicos
        if token == 'fins': continue

        #get list of docs
        docs = list(melted_df[melted_df['token'] == token]['docs'])
        
        temp_dict = {'token': token, 'score': score, 'docs': docs}
        tokens_list.append(temp_dict)
    file_path = outputs+'tokens_list_{}.json'.format(str(i))
    json.dump(tokens_list, open(file_path, 'w'))